In [1]:
import pyspark
from delta import *
import logging

builder = pyspark.sql.SparkSession.builder.appName("Delta") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

21/09/28 14:02:06 WARN Utils: Your hostname, DESKTOP-C98VC8U resolves to a loopback address: 127.0.1.1; using 172.22.212.21 instead (on interface eth0)
21/09/28 14:02:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/wellington/.ivy2/cache
The jars for the packages stored in: /home/wellington/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-822eb392-45f7-43a6-9b18-68a1eaeed747;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 1198ms :: artifacts dl 63ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#

In [3]:
from pyspark.sql.types import *
schema1 = StructType([
                        StructField("ticket_id", IntegerType(), True),
                        StructField("order_id", IntegerType(), True),
                        StructField("customer_id", IntegerType(), True),
                        StructField("quantity", IntegerType(), True),
                        StructField("net_sales", FloatType(), True),
                        StructField("event_code", StringType(), True),
                        StructField("event_name", StringType(), True),
                        StructField("Event_date", DateType(), True),
                        StructField("event_season", StringType(), True)]
                        
                    )

In [4]:
deltaTable = DeltaTable.convertToDelta(spark, "parquet.`/mnt/c/DataSets/tickets.parquet`") 

The table you are trying to convert is already a delta table


In [6]:
df = spark.read.format("delta").load("/mnt/c/DataSets/tickets.parquet")
df.show(3)

+---------+--------+-----------+--------+------------------+----------+------------------+----------+------------+----------+
|ticket_id|order_id|customer_id|quantity|         net_sales|event_code|        event_name|event_date|event_season|      Date|
+---------+--------+-----------+--------+------------------+----------+------------------+----------+------------+----------+
|        1|     182|        160|       1|58.549109713360856|   CHL-ARS|Chelsea vs Arsenal|5-May-2018|   2020/2021|2018-05-05|
|        2|     376|         46|       1|44.254019825925596|   CHL-ARS|Chelsea vs Arsenal|5-May-2018|   2020/2021|2018-05-05|
|        3|     282|        116|       5|47.003734818026274|   CHL-ARS|Chelsea vs Arsenal|5-May-2018|   2020/2021|2018-05-05|
+---------+--------+-----------+--------+------------------+----------+------------------+----------+------------+----------+
only showing top 3 rows



In [7]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
cust1 = spark.read\
             .option("dateFormat", "dd-MMM-yyyy")\
             .option("sep", "|")\
             .option("header", True)\
             .schema(schema1)\
             .csv("./data/tickets.csv")
  
cust1.show(3)

ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8

+---------+--------+-----------+--------+---------+----------+------------------+----------+------------+
|ticket_id|order_id|customer_id|quantity|net_sales|event_code|        event_name|Event_date|event_season|
+---------+--------+-----------+--------+---------+----------+------------------+----------+------------+
|        1|     182|        160|       1| 58.54911|   CHL-ARS|Chelsea vs Arsenal|2018-05-05|   2020/2021|
|        2|     376|         46|       1| 44.25402|   CHL-ARS|Chelsea vs Arsenal|2018-05-05|   2020/2021|
|        3|     282|        116|       5|47.003735|   CHL-ARS|Chelsea vs Arsenal|2018-05-05|   2020/2021|
+---------+--------+-----------+--------+---------+----------+------------------+----------+------------+
only showing top 3 rows



In [8]:
cust2 = spark.read\
             .option("dateFormat", "yyyy-MMM-dd")\
             .option("sep", "|")\
             .option("header", True)\
             .schema(schema1)\
             .csv("./data/tickets.csv")
cust2.show(3)

+---------+--------+-----------+--------+---------+----------+------------------+----------+------------+
|ticket_id|order_id|customer_id|quantity|net_sales|event_code|        event_name|Event_date|event_season|
+---------+--------+-----------+--------+---------+----------+------------------+----------+------------+
|        1|     182|        160|       1| 58.54911|   CHL-ARS|Chelsea vs Arsenal|0010-11-08|   2020/2021|
|        2|     376|         46|       1| 44.25402|   CHL-ARS|Chelsea vs Arsenal|0010-11-08|   2020/2021|
|        3|     282|        116|       5|47.003735|   CHL-ARS|Chelsea vs Arsenal|0010-11-08|   2020/2021|
+---------+--------+-----------+--------+---------+----------+------------------+----------+------------+
only showing top 3 rows



In [9]:
import pyspark.sql.functions as F
print(cust1.select("Event_date").where(F.col("Event_date").isNotNull()).count(), 
      cust2.select("Event_date").where(F.col("Event_date").isNotNull()).count()
      )

667 667


In [10]:
cust1.printSchema()

root
 |-- ticket_id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- net_sales: float (nullable = true)
 |-- event_code: string (nullable = true)
 |-- event_name: string (nullable = true)
 |-- Event_date: date (nullable = true)
 |-- event_season: string (nullable = true)



In [14]:
customers = spark.read.parquet("./output/customers.parquet")
customers.show(3)

+----------------+---------+--------+--------------------+--------+--------------+
|CustomerIdentity|FirstName|LASTNAME|               phone|postcode|Customer_Title|
+----------------+---------+--------+--------------------+--------+--------------+
|               1|    Terri| Leonard|          3124065463|   65369|          Mrs.|
|               2|  Maurice|    West|001-468-075-5301x308|   27061|           Dr.|
|               3|   Taylor|    Hall|    001-004-938-2651|   88302|           Mr.|
+----------------+---------+--------+--------------------+--------+--------------+
only showing top 3 rows



In [11]:
tickets = spark.read.parquet("./output/tickets.parquet")
tickets.show(3)

+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+
|ticket_id|order_id|customer_id|quantity|Net_sales|event_code|        event_name|event_season|      Date|
+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+
|        1|     182|        160|       1|    58.55|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|
|        2|     376|         46|       1|    44.25|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|
|        3|     282|        116|       5|     47.0|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|
+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+
only showing top 3 rows



In [12]:
tickets.printSchema()

root
 |-- ticket_id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- Net_sales: double (nullable = true)
 |-- event_code: string (nullable = true)
 |-- event_name: string (nullable = true)
 |-- event_season: string (nullable = true)
 |-- Date: date (nullable = true)



In [15]:
#broadcast joining because one table is very small, reduces shuffling
import pyspark.sql.functions as F
join_condition = customers.CustomerIdentity == tickets.customer_id
combined_df =  tickets.join(F.broadcast(customers), join_condition).drop("CustomerIdentity")
combined_df.cache()

DataFrame[ticket_id: int, order_id: int, customer_id: int, quantity: int, Net_sales: double, event_code: string, event_name: string, event_season: string, Date: date, FirstName: string, LASTNAME: string, phone: string, postcode: string, Customer_Title: string]

In [16]:
print(len(combined_df.columns), combined_df.count())

14 1000


In [17]:
combined_df.select("Date").where(F.col("Date").isNull()).count()

0

In [19]:
combined_df.show(3, truncate=True)

+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+---------+--------+--------------------+--------+--------------+
|ticket_id|order_id|customer_id|quantity|Net_sales|event_code|        event_name|event_season|      Date|FirstName|LASTNAME|               phone|postcode|Customer_Title|
+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+---------+--------+--------------------+--------+--------------+
|        1|     182|        160|       1|    58.55|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|     Tony|   Baird|001-097-596-0629x...|   94414|           Ms.|
|        2|     376|         46|       1|    44.25|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|   Justin| Higgins|          1889149702|   67579|          Mrs.|
|        3|     282|        116|       5|     47.0|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05| Jennifer|  Dawson|    479.687.6222x409|   23

In [20]:
combined_df.select("customer_id").distinct().count()

198

In [21]:
combined_df.createOrReplaceTempView("DF")

In [22]:
spark.sql("""
          SELECT Date, order_id
          FROM DF 
          WHERE Date == "2018-01-30"
          """).show(2)

+----------+--------+
|      Date|order_id|
+----------+--------+
|2018-01-30|      20|
|2018-01-30|      14|
+----------+--------+
only showing top 2 rows



In [27]:
cols1 = ['order_id', 'ticket_id', 'customer_id', 'event_code', 'Date',  'quantity', 'Net_sales']
transcation = combined_df.select(cols1)\
                         .distinct()\
                         .orderBy("Date")
transcation.show(3)

+--------+---------+-----------+----------+----------+--------+---------+
|order_id|ticket_id|customer_id|event_code|      Date|quantity|Net_sales|
+--------+---------+-----------+----------+----------+--------+---------+
|     219|      188|        132|   F1-SILV|2018-01-30|       2|    28.82|
|     342|      207|        180|   F1-SILV|2018-01-30|       5|    25.15|
|       2|      838|         95|   BTH-SRC|2018-01-30|       2|     84.0|
+--------+---------+-----------+----------+----------+--------+---------+
only showing top 3 rows



In [28]:
event = combined_df.select("event_code", "event_name", "event_season").distinct()
event.show()

+----------+------------------+------------+
|event_code|        event_name|event_season|
+----------+------------------+------------+
|   BTH-SRC|  Bath vs Saracens|   2020/2021|
|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|
|   F1-SILV|       Silverstone|   2020/2021|
+----------+------------------+------------+



In [15]:
customer = combined_df.select("customer_id", "Customer_Title", "FirstName", "LASTNAME", "phone", "postcode").distinct().orderBy("customer_id")
customer.show(3)

+-----------+--------------+---------+--------+--------------------+--------+
|customer_id|Customer_Title|FirstName|LASTNAME|               phone|postcode|
+-----------+--------------+---------+--------+--------------------+--------+
|          1|          Mrs.|    Terri| Leonard|          3124065463|   65369|
|          2|           Dr.|  Maurice|    West|001-468-075-5301x308|   27061|
|          3|           Mr.|   Taylor|    Hall|    001-004-938-2651|   88302|
+-----------+--------------+---------+--------+--------------------+--------+
only showing top 3 rows



In [16]:
import pyspark.sql.functions as F
# 3. For each Customer, a list of Events
Edf = combined_df.groupBy("customer_id")\
                 .agg(F.collect_list(F.col("event_code"))\
                 .alias("Events_list"))
Edf.show(3, truncate=False)

+-----------+------------------------------------------------------------------------------------------+
|customer_id|Events_list                                                                               |
+-----------+------------------------------------------------------------------------------------------+
|148        |[BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV, CHL-ARS]                                             |
|31         |[CHL-ARS, CHL-ARS, BTH-SRC, BTH-SRC, CHL-ARS, CHL-ARS, BTH-SRC, CHL-ARS, BTH-SRC, CHL-ARS]|
|85         |[CHL-ARS, F1-SILV, F1-SILV]                                                               |
+-----------+------------------------------------------------------------------------------------------+
only showing top 3 rows



In [17]:
# combined_df.groupBy("CustomerIdentity").agg(F.count("event_code").alias("Totals")).sort("Totals", ascending=True).show()
Tots = combined_df.groupBy("customer_id").agg(F.count("event_name").alias("Totals"))
Tots.show(3)

+-----------+------+
|customer_id|Totals|
+-----------+------+
|        148|     5|
|         31|    10|
|        137|     7|
+-----------+------+
only showing top 3 rows



In [18]:
MultiEvent = Tots.withColumn("MultiEvent", F.col("Totals") > 1).drop("Totals")
MultiEvent.show(6)

+-----------+----------+
|customer_id|MultiEvent|
+-----------+----------+
|        148|      true|
|         31|      true|
|        137|      true|
|         85|      true|
|         65|      true|
|         53|     false|
+-----------+----------+
only showing top 6 rows



In [87]:
dfff = combined_df.select("Date", "customer_id", "ticket_id", "Net_sales")
dfff.show(3)

+----------+-----------+---------+---------+
|      Date|customer_id|ticket_id|Net_sales|
+----------+-----------+---------+---------+
|2018-05-05|        160|        1|    58.55|
|2018-05-05|         46|        2|    44.25|
|2018-05-05|        116|        3|     47.0|
+----------+-----------+---------+---------+
only showing top 3 rows



In [83]:
from pyspark.sql.window import Window
cols = ["Date", "customer_id", "ticket_id", "quantity", "Net_sales"]

windowSpec = Window.partitionBy(["customer_id"])\
                    .orderBy("Date")\
                    .rowsBetween(-1, Window.currentRow)
res = combined_df.select(cols).withColumn("RunningTotal", F.round(F.sum(F.col("quantity")).over(windowSpec), 2))
res.show()

+----------+-----------+---------+--------+---------+------------+
|      Date|customer_id|ticket_id|quantity|Net_sales|RunningTotal|
+----------+-----------+---------+--------+---------+------------+
|2018-01-30|        148|      347|       3|    38.18|           3|
|2018-01-30|        148|      789|       4|    60.01|           7|
|2018-05-05|        148|       84|       3|    66.54|           7|
|2018-05-05|        148|      376|       3|    94.75|           6|
|2018-05-05|        148|      574|       2|    97.67|           5|
|2018-01-30|         31|       30|       5|    67.23|           5|
|2018-01-30|         31|      158|       3|    37.92|           8|
|2018-01-30|         31|      619|       3|    20.32|           6|
|2018-01-30|         31|      778|       4|     4.86|           7|
|2018-05-05|         31|       40|       5|    56.49|           9|
|2018-05-05|         31|      208|       5|    68.67|          10|
|2018-05-05|         31|      329|       1|    69.37|         

In [77]:
res.collect()[1]["customer_id"]

148

In [82]:
# 7. Gap/Delta in Quantity between each Customers Order

from pyspark.sql.window import Window
from pyspark.sql.functions import lag 
cols = ["Date", "customer_id", "ticket_id", "quantity"]
dfff = combined_df.select(cols)
windowSpec = Window.partitionBy(["customer_id"])\
                    .orderBy("Date")
                    
r = dfff.withColumn("lag",lag("quantity", 1).over(windowSpec)) \
      .withColumn("delta", F.round(-F.col("lag")+F.col("quantity")))
      
r.na.fill(value=0).show(100)

+----------+-----------+---------+--------+---+-----+
|      Date|customer_id|ticket_id|quantity|lag|delta|
+----------+-----------+---------+--------+---+-----+
|2018-01-30|        148|      347|       3|  0|    0|
|2018-01-30|        148|      789|       4|  3|    1|
|2018-05-05|        148|       84|       3|  4|   -1|
|2018-05-05|        148|      376|       3|  3|    0|
|2018-05-05|        148|      574|       2|  3|   -1|
|2018-01-30|         31|       30|       5|  0|    0|
|2018-01-30|         31|      158|       3|  5|   -2|
|2018-01-30|         31|      619|       3|  3|    0|
|2018-01-30|         31|      778|       4|  3|    1|
|2018-05-05|         31|       40|       5|  4|    1|
|2018-05-05|         31|      208|       5|  5|    0|
|2018-05-05|         31|      329|       1|  5|   -4|
|2018-05-05|         31|      581|       3|  1|    2|
|2018-05-05|         31|      853|       4|  3|    1|
|2018-05-05|         31|      901|       5|  4|    1|
|2018-01-30|         85|    

#List of **all** Customers with an additional column called "MultiEvent", set to `True` for those Customers with more than 1 Event
Tots.withColumn("MultiEvent", multiEvent_udf(F.col("Totals"))).show(5)

In [29]:
cols = ["customer_id", "event_name", "event_season", "Date", "rank"]
combined_df.groupBy("customer_id").agg(F.max("quantity")).show()

+-----------+-------------+
|customer_id|max(quantity)|
+-----------+-------------+
|        148|            4|
|         31|            5|
|        137|            5|
|         85|            5|
|         65|            4|
|         53|            2|
|        133|            5|
|         78|            4|
|        108|            5|
|        155|            5|
|         34|            4|
|        193|            5|
|        115|            5|
|        101|            5|
|        126|            5|
|         81|            3|
|        183|            5|
|         28|            3|
|         76|            5|
|         26|            5|
+-----------+-------------+
only showing top 20 rows



In [44]:
# 5. Largest Order by Quantity for each Customer
from pyspark.sql.window import Window
cols = ["customer_id", "quantity"]
windowSpec1 = Window.partitionBy(["customer_id"])\
                    .orderBy(F.desc("quantity"))

ranked = combined_df.withColumn("rank", F.rank().over(windowSpec1)).filter("rank == 1").select(cols)
ranked.show(5)

+-----------+--------+
|customer_id|quantity|
+-----------+--------+
|        148|       4|
|         31|       5|
|         31|       5|
|         31|       5|
|         31|       5|
+-----------+--------+
only showing top 5 rows



In [50]:
ranked.drop_duplicates()\
        .orderBy(F.desc("quantity"))\
        .orderBy("customer_id").show()

+-----------+--------+
|customer_id|quantity|
+-----------+--------+
|          1|       3|
|          2|       4|
|          3|       5|
|          4|       5|
|          5|       4|
|          6|       5|
|          7|       5|
|          8|       5|
|          9|       5|
|         10|       5|
|         11|       4|
|         12|       3|
|         13|       4|
|         14|       3|
|         15|       2|
|         16|       5|
|         17|       4|
|         18|       2|
|         19|       5|
|         20|       5|
+-----------+--------+
only showing top 20 rows



In [30]:
from pyspark.sql import Window

WindowSpec = Window.partitionBy("customer_id").orderBy(F.desc("quantity"))

F = combined_df.withColumn("dddd", F.collect_list(F.col("quantity")).over(WindowSpec)).select("customer_id", "dddd")
F.show()

+-----------+--------------------+
|customer_id|                dddd|
+-----------+--------------------+
|        148|                 [4]|
|        148|        [4, 3, 3, 3]|
|        148|        [4, 3, 3, 3]|
|        148|        [4, 3, 3, 3]|
|        148|     [4, 3, 3, 3, 2]|
|         31|        [5, 5, 5, 5]|
|         31|        [5, 5, 5, 5]|
|         31|        [5, 5, 5, 5]|
|         31|        [5, 5, 5, 5]|
|         31|  [5, 5, 5, 5, 4, 4]|
|         31|  [5, 5, 5, 5, 4, 4]|
|         31|[5, 5, 5, 5, 4, 4...|
|         31|[5, 5, 5, 5, 4, 4...|
|         31|[5, 5, 5, 5, 4, 4...|
|         31|[5, 5, 5, 5, 4, 4...|
|         85|                 [5]|
|         85|              [5, 4]|
|         85|           [5, 4, 1]|
|        137|              [5, 5]|
|        137|              [5, 5]|
+-----------+--------------------+
only showing top 20 rows



In [ ]:
F.collect()[2]["dddd"]

In [22]:
from pyspark.sql import Window

WindowSpec = Window.partitionBy("customer_id")

df22 = combined_df.withColumn("EventsList", 
                              F.collect_list(F.col("event_code"))
                              .over(WindowSpec)
                              )\
.select("customer_id", "EventsList")
df22.show(5, truncate=False)



+-----------+---------------------------------------------+
|customer_id|EventsList                                   |
+-----------+---------------------------------------------+
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
+-----------+---------------------------------------------+
only showing top 5 rows



In [16]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="event_code", outputCol="IndexEvents")
indexed = indexer.fit(combined_df).transform(combined_df)
indexed.show(10)

+---------+--------+--------+---------+----------+------------------+------------+----------+----------------+---------+--------+--------------------+--------+--------------+-----------+
|ticket_id|order_id|quantity|Net_sales|event_code|        event_name|event_season|      Date|CustomerIdentity|FirstName|LASTNAME|               phone|postcode|Customer_Title|IndexEvents|
+---------+--------+--------+---------+----------+------------------+------------+----------+----------------+---------+--------+--------------------+--------+--------------+-----------+
|        1|     182|       1|    58.55|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|             160|     Tony|   Baird|001-097-596-0629x...|   94414|           Ms.|        0.0|
|        2|     376|       1|    44.25|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|              46|   Justin| Higgins|          1889149702|   67579|          Mrs.|        0.0|
|        3|     282|       5|     47.0|   CHL-ARS|Chelsea vs Arse

In [18]:
combined_df.groupBy("Customer_Title").agg(F.sum("quantity").alias("Totals")).sort("Totals", ascending=False).show()

+--------------+------+
|Customer_Title|Totals|
+--------------+------+
|           Dr.|  1023|
|           Mr.|   996|
|          Mrs.|   691|
|          Miss|   131|
|           Ms.|   128|
+--------------+------+



In [27]:
add = lambda x,y : x+y
add(2,3)

5

In [15]:
import sys

In [98]:

#List of **all** Customers with an additional column called "MultiEvent", set to `True` for those Customers with more than 1 Event
combined_df.groupBy("customer_id")\
           .agg(F.count("event_code").alias("Totals"))\
           .sort("Totals", ascending=True)\
           .show()

+-----------+------+
|customer_id|Totals|
+-----------+------+
|         86|     1|
|        175|     1|
|        121|     1|
|        126|     1|
|         28|     1|
|         53|     1|
|        149|     1|
|        109|     2|
|         61|     2|
|         62|     2|
|        117|     2|
|        154|     2|
|         24|     2|
|        156|     2|
|          5|     2|
|          1|     2|
|         15|     2|
|        171|     2|
|         91|     2|
|         81|     2|
+-----------+------+
only showing top 20 rows

